In [1]:
from keras.models import Sequential, load_model
from keras.layers import Input, Dense, Dropout
from tcn import TCN
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import ParameterGrid

In [2]:
# Load Data
X_train = np.load('X_train.npy')  # Load your preprocessed training data
y_train = np.load('y_train.npy')
X_val = np.load('X_val.npy')  # Load your validation data
y_val = np.load('y_val.npy')
X_test = np.load('X_test.npy')  # Load your test data
y_test = np.load('y_test.npy')

In [3]:
# Model Parameters
num_classes = len(np.unique(y_train))

In [4]:
# Define the parameter grid
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10],
    'dropout_rate': [0.2, 0.3],
    'filters': [32, 64],
    'kernel_size': [3, 5]
}

In [5]:
# Create a list of all parameter combinations
param_combinations = list(ParameterGrid(param_grid))
print(f"Total combinations to try: {len(param_combinations)}")

# Results storage
results = []
best_val_acc = 0
best_model = None  # To store the best model

Total combinations to try: 16


In [6]:
for params in param_combinations:
    print(f"\nTraining with parameters: {params}")

    # Build the model with hyperparameters from the grid search
    model = Sequential()

    # Add Input layer explicitly
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))  # Input shape is specified here

    # Add TCN layers
    model.add(TCN(nb_filters=params["filters"], kernel_size=params["kernel_size"]))
    model.add(Dropout(params["dropout_rate"]))  # Regularization
    model.add(Dense(32, activation='relu'))  # Smaller Dense layer
    model.add(Dense(num_classes, activation='softmax'))  # Output layer for classification

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model with the specified batch size and number of epochs
    model.fit(X_train, y_train, epochs=params["epochs"], batch_size=params["batch_size"],
              validation_data=(X_val, y_val))

    # Evaluate the model on the validation set
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy: {val_acc:.4f}")

    # If this is the best model so far, save it
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = model  # Store the best model

        # Save the best model
        model.save("Saved Models/best_tcn.keras")
        print(f"Best model saved with validation accuracy: {best_val_acc:.4f}")


Training with parameters: {'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 10, 'filters': 32, 'kernel_size': 3}

Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 19s 91ms/step - accuracy: 0.6610 - loss: 274.3036 - val_accuracy: 0.8501 - val_loss: 38.7467
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 15s 91ms/step - accuracy: 0.7734 - loss: 46.5629 - val_accuracy: 0.8530 - val_loss: 27.6583
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 15s 88ms/step - accuracy: 0.8189 - loss: 26.9558 - val_accuracy: 0.8646 - val_loss: 21.5728
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step - accuracy: 0.7901 - loss: 21.4765 - val_accuracy: 0.8607 - val_loss: 10.5712
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 12s 73ms/step - accuracy: 0.8105 - loss: 15.5685 - val_accuracy: 0.8761 - val_loss: 10.9619
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - accuracy: 0.8133 - loss: 13.1499 - val_accuracy: 0.7992 - val_loss: 12.6824
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step - accuracy: 0.8273 - loss: 9.9663 - va

In [7]:
#After tuning, load the best model
loaded_model = load_model('best_tcn.keras')

# Predict and evaluate on the test set using the loaded model
y_pred = np.argmax(loaded_model.predict(X_test), axis=-1)
print("Classification Report:")
print(classification_report(y_test, y_pred))

27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 166ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.90      0.92       539
           1       0.83      0.90      0.86       298

    accuracy                           0.90       837
   macro avg       0.89      0.90      0.89       837
weighted avg       0.90      0.90      0.90       837

